# PROYECTO ISIS-3302
# ITERACIÓN 1

## 1. Lectura y Comprensión del Enunciado

El Proyecto A plantea un problema de logística urbana en la ciudad de Bogotá enfocado en la distribución eficiente de mercancías. La empresa LogistiCo, a través de su división Urban Logistics, opera múltiples centros de distribución en la ciudad y sus alrededores para hacer frente al crecimiento de las entregas de comercio electrónico​. El objetivo fundamental es minimizar los costos operativos de transporte (combustible, mantenimiento, etc.) garantizando entregas rápidas y seguras a todos los clientes, y cumpliendo con las restricciones físicas y organizacionales del negocio. En Bogotá, esto implica enfrentar desafíos importantes como la elevada congestión vehicular en varios puntos de la ciudad, restricciones de acceso debido a obras y exigentes tiempos de servicio.

Para comprender el enunciado, se identifican los elementos del sistema de envíos de LogistiCo en Bogotá y se abstrae la situación a un problema de optimización combinatoria. 

Lo que se quiere hacer es planificar qué centro de distribución atiende a cada pedido y con qué vehículo y trazar las rutas óptimas de los vehículos para cumplir con todas las entregas sobre el mapa de la ciudad. Este problema corresponde a un problema de enrutamiento de vehículos en logística (un VRP por sus siglas en inglés, Vehicle Routing Problem), con particularidades como múltiples depósitos (centros de distribución), una flota heterogénea con capacidades y autonomías limitadas, y posibles múltiples paradas por ruta. 

Ahora bien, cada ruta de vehículo se asemeja a un problema de agente viajero (TSP, Travelling Salesman Problem) extendido semejante al Problema 5 del Laboratorio 2, en el que se cuenta con varios agentes. En este caso, varios vehículos que deben seguir rutas a través de varios depósitos y varios puntos de entrega según su demanda respectiva. Por lo tanto, el problema puede interpretarse como un VRP con multiples depósitos y con capacidades, donde se debe decidir la asignación de clientes a centros de distribución y la secuencia de visitas de cada vehículo, minimizando el costo total.

## 2. Identificación de Elementos Clave

A partir del enunciado, se distinguen los **componentes esenciales** del sistema logístico y las **limitaciones** que lo rigen:

- **Centros de Distribución (CD)**: Instalaciones desde donde parten los vehículos con los pedidos. En el ejemplo se cuentan con tres centros (CD1: Bodega Norte, CD2: Bodega Sur, CD3: Bodega Este) con ubicaciones geográficas dadas (latitud, longitud) y una capacidad máxima de almacenamiento en cada uno. La capacidad de cada centro (por ejemplo, 20,000 kg en CD1 ) es una restricción física que limita la cantidad de inventario que se puede gestionar desde allí. Esto significa que la **asignación de inventario** a cada bodega debe respetar su límite para no exceder el espacio disponible.

- **Clientes o Zonas de Entrega (C)**: Son los destinos finales de los pedidos. Cada cliente tiene una ubicación geográfica (latitud, longitud) y una demanda asociada (cantidad de unidades a entregar). Por ejemplo, en los datos de muestra se tienen tres clientes (C1: Catalina, C2: Rodrigo, C3: Luis) con demandas de 50, 80 y 65 unidades respectivamente. Todos los clientes deben ser atendidos, es decir, **cada demanda debe ser satisfecha por completo** por al menos un vehículo. Una limitación organizacional importante es el nivel de servicio: se espera cumplir todas las entregas en tiempo y forma, lo que implica que no se pueden dejar demandas sin atender (o habría un costo de penalización muy alto por incumplimiento).

- **Vehículos (V)**: La flota vehicular asignada a las entregas urbanas. En el ejemplo hay tres vehículos disponibles (V1, V2, V3), cada uno con **capacidad máxima de carga** (e.g. 100, 80 y 150 unidades respectivamente) y **autonomía de recorrido** en kilómetros (120, 100 y 150 km). Estas cifras imponen restricciones físicas en el modelo: un vehículo no puede transportar más mercancía de la que permite su capacidad y no puede recorrer más distancia de la que le otorga su autonomía de combustible o batería. La **gestión de vehículos** deberá entonces respetar que la suma de demandas atendidas por un vehículo no exceda su capacidad, y que la distancia total recorrida en su ruta no exceda su rango útil.

- **Red de rutas (vías urbanas)**: Conecta geográficamente los centros de distribución con los clientes. A diferencia de un plano euclideano simple, la red vial impone distancias reales y tiempos de viaje que pueden ser mayores que la distancia en línea recta entre dos puntos debido a la estructura de las calles. Un elemento clave es la **planificación de rutas**: los vehículos pueden visitar múltiples clientes en un mismo recorrido, realizando una ruta que conecta un centro con varios clientes de forma secuencial. Esto introduce decisiones sobre qué secuencia de visitas realiza cada vehículo para optimizar el recorrido general (por ejemplo, un mismo camión podría entregar primero al cliente A y luego al B antes de regresar al depósito). La posibilidad de **conexiones directas entre clientes en una misma ruta** se reconoce como una oportunidad para mejorar la eficiencia, en lugar de regresar al centro de distribución tras cada entrega individual. Sin embargo, también se debe asegurar que no queden “subrutas” aisladas (subtours) que no conecten con un centro, respetando la conectividad de cada ruta.

- **Costos operativos**: Si bien no son un componente tangible, los costos (combustible, mantenimiento, personal, etc.) son un elemento clave a optimizar. El enunciado menciona tarifas de flete por kilómetro, costos de combustible por litro y costos de mantenimiento por kilómetro, entre otros. Estos parámetros económicos se deben **identificar y cuantificar** adecuadamente para incorporarlos en el modelo. Limitaciones organizacionales como el presupuesto disponible o los niveles de servicio mínimos podrían también considerarse, pero principalmente estos costos se integran en el objetivo a minimizar.

Además de estos componentes, se identifican las **limitaciones inherentes al negocio**: la integridad del inventario (no sobrepasar capacidades de bodega), las restricciones físicas de los vehículos (capacidad y autonomía), la necesidad de cumplir todas las entregas (restricción de cobertura de demanda), y las condiciones urbanas (congestión, posibles ventanas de tiempo de entrega, etc., aunque estas últimas no se detallan en el enunciado, podrían extender el modelo). En resumen, el sistema se compone de múltiples depósitos con inventario limitado, una flota de vehículos limitada en carga y distancia, y un conjunto de clientes con demandas, todo ello conectado por posibles rutas en la red urbana. Estas características y restricciones delinean el marco dentro del cual se formulará el modelo de optimización.


## 2. Formulación de la Estructura del Modelo:

### A continuación se presentan los conjuntos,  parámetros, variables de decisión y restricciones que conforman la formulación matemática del problema. 

### 2.1 CONJUNTOS

- I: Conjunto de Centros de Distribución = {1, 2, ..., m}. En el ejemplo, I = {CD1, CD2, CD3}.

- J: Conjunto de Clientes (o puntos de entrega) = {1, 2, ..., n}. En el ejemplo, J = {C1, C2, C3}.

- V: Conjunto de Vehículos disponibles = {1, 2, ..., k}. En el ejemplo, V = {V1, V2, V3}.

Para modelar las rutas, definimos el conjunto N de nodos que incluye todos los puntos 
involucrados en las rutas. En este modelo 
asumiremos que salir de un CD y volver al mismo CD se modela con N = I U J y 
controlaremos inicios y fines con restricciones.

- A: Conjunto de arcos posibles en la red logística. Generalmente, A = {(u, v): u, v ∈ N, u ≠ v}, 
es decir, todos los pares de nodos distintos. En este caso, los arcos 
representan tramos de ruta posibles entre un origen u y un destino v. Asumimos que los vehículos no 
intercambian carga entre CDs durante la ruta. De esta manera, las rutas son circuitos que inician y 
terminan en el mismo CD.

### 2.2 PaRÁMETROSrámetrosPA

- $A_i$: Capacidad de almacenamiento del centro de distribución $i \in I$ (en unidades de producto). Por ejemplo, $A_{\text{CD1}} = 20000$ kg 
kg y las *unidades* de demanda; asumiremI
SIS_3302_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=ID%20Cliente%20Zona%20Latitud%20Longitud,unidades)).
- $Q_k$: Capacida
ISIS_3302_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=ID%20Veh%C3%ADculo%20Cap ([
ISIS_3302_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=ID%20Veh%C3%ADculo%20Capacidad%20%C3%9Atil%20,km)).
- $c_{uv}$: **Costo unitario de recorrer** el arco desde nodo $u$ hasta nodo $v$ (por lo general en COP, pesos colombianos). Este costo puede descomponerse o calcularse a 302_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=Pf%20Precio%20del%20combustible%20,por%20km%29%20%24700)). En este proyecto se propone calcular $c_{uv}$ principalmente como un costo proporcional a la 302_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=Pf%20Precio%20del%20combustible%20,por%20km%29%20%24700)). Po02_2025_10_PROYECTO_proyecto_1.pdf](file://file-M6Bf6EbqW1JgtAFKYhAR4t#:~:text=Pf%20Precio%20del%20combustible%20,por%20km%29%20%24700))), y la distancia en ruta entre dos puntos es 10 km, entonces $c_{uv} \approx 20700 \times 10 = 2070
$ COP. (Para simplificar, supondremos costos lineales con la distancia y usaremos la distancia como proxy del costo en la formulación.)

- $d_{uv}$: **Distancia** (en km) por la ruta óptima de la red vial desde el nodo $u \in N$ hasta el nodo $v \in N$. Este parámetro se obtiene en la etapa de preprocesamiento de datos usando herramientas GIS, y es fundamental para el cálculo de costos y verificación de autonomías.


### 2.3 VARIABLES DE DECISIÓN

- $x_{uvk} \in \{0,1\}$: Variable binaria que vale 1 si el **vehículo** $k \in K$ viaja directamente desde el nodo $u \in N$ hasta el nodo $v \in N$ en su ruta, y 0 en caso contrario. Estas variables definen la **ruta recorrida por cada vehículo**, indicando qué arcos del grafo de nodos se utilizan. En la solución, una ruta completa de un vehículo corresponde a una secuencia de arcos $x_{uv,k}=1$ que forman un circuito: inicia en un centro $u \in I$, pasa por ciertos clientes intermedios, y termina retornando a un centro (posiblemente el mismo con el que inició). 

- $y_{ik} \in \{0,1\}$: Variable binaria que vale 1 si el vehículo $k$ inicia su ruta desde el centro de distribución $i$. Esta variable puede ser útil en un contexto de **múltiples depósitos** para indicar qué centro (depósito) “usa” cada vehículo. En muchos casos se conoce de antemano qué vehículos pertenecen a qué centro; si no, $y_{ik}$ permitiría decidirlo en el modelo. En nuestro ejemplo asumiremos que cada vehículo ya está adscrito a un centro específico (por simplicidad, supongamos V1 parte de CD1, V2 de CD2, V3 de CD3), por lo que $y_{ik}$ sería dato fijo en 1 para esas combinaciones y 0 para las demás.

- $u_{j,k} \ge 0$: Variables continuas auxiliares para cada cliente $j$ y vehículo $k$, utilizadas para eliminar *subtours*. Estas variables suelen representar el orden de visita de los clientes en la ruta de un vehículo (p. ej., $u_{j,k}$ puede indicar la posición del cliente $j$ en la ruta del vehículo $k$).

### 2.4 FUNCIÓN OBJETIVO

**Función Objetivo (FO)** del modelo busca minimizar el costo total de operación, típicamente asociado a la suma de distancias recorridas por todos los vehículos (ponderadas por el costo por kilómetro). Matemáticamente, se puede expresar así:

$$
\min \; Z \;=\; \sum_{k \in K} \sum_{u \in N} \sum_{v \in N} c_{uv}\; x_{uvk}
$$

donde $c_{uv}$ incorpora los costos por kilómetro y $x_{uvk}$ indica si ese kilómetro se recorre o no. De este modo, $Z$ suma el costo de todos los trayectos realizados por todos los vehículos. En ausencia de costos fijos por uso de vehículos, esta FO equivaldría a minimizar la **distancia total recorrida** por la flota. (Si se quisieran incluir otros objetivos, como minimizar el número de vehículos usados, se podrían añadir términos o penalizaciones correspondientes. Sin embargo, aquí consideramos suficiente la minimización del costo/distancia, ya que un menor recorrido tenderá también a usar menos vehículos de forma indirecta.)

### 2.5 RESTRICCIONES


1. **Cobertura de clientes:** cada cliente $j$ debe ser visitado **exactamente una vez** por **exactamente un vehículo**. Esto se asegura con la restricción:

   $$
   \sum_{k \in K} \sum_{u \in N} x_{u,j,k} = 1, \qquad \forall j \in J\,,
   $$

   que indica que la suma de arcos que *entran* al cliente $j$ (provenientes de algún nodo $u$, sea un centro u otro cliente, por algún vehículo $k$) debe ser 1. Equivalentemente, cada cliente tiene un único predecesor en alguna ruta de algún vehículo. Esta condición garantiza que **todos los pedidos son atendidos** y que no haya duplicación (un mismo cliente visitado dos veces o por dos vehículos distintos). 

2. **Conservación de flujo (continuidad de ruta):** para cada cliente que es atendido por un vehículo, la llegada y salida de ese cliente en la ruta deben estar conectadas. Esto se formula exigiendo que, si un vehículo $k$ entra al nodo $j$, también debe salir de $j$ (a menos que $j$ sea la última visita y se retorne al depósito). Matemáticamente, para cada cliente $j$ y cada vehículo $k$:

   $$
   \sum_{u \in N} x_{u,j,k} = \sum_{v \in N} x_{j,v,k}, \qquad \forall j \in J,\; \forall k \in K\,.
   $$

   Esta ecuación balancea el arco de entrada y el arco de salida del cliente $j$ en la ruta del vehículo $k$. Garantiza que la ruta de $k$ que pasa por $j$ no se interrumpe allí (lo que impediría completar el circuito). Para los centros de distribución, la condición es ligeramente distinta: cada vehículo que parte de un centro $i$ debe también regresar a (posiblemente el mismo) un centro al final. Si asumimos que cada vehículo debe retornar a su centro de origen, entonces para cada vehículo $k$ asignado al centro $i$ habría:

   $$
   \sum_{v \in N} x_{i,v,k} = \sum_{u \in N} x_{u,i,k} = 1, \qquad \forall k \text{ que sale de depot } i\,,
   $$

   lo cual indica que el vehículo $k$ sale una vez del depósito $i$ (iniciando su ruta) y regresa una vez a ese mismo depósito al finalizar (ciclo cerrado). Esta es la formalización de que cada vehículo realiza **una única ruta que inicia y termina en su centro asignado**.

3. **Capacidad de vehículos:** la suma de las demandas atendidas por un vehículo $k$ en su recorrido no debe exceder su capacidad $Q_k$. Para implementar esta restricción, se pueden usar las variables $x$ sumadas sobre los clientes: 

   $$
   \sum_{j \in J} d_j \sum_{u \in N} x_{u,j,k} \le Q_k, \qquad \forall k \in K\,.
   $$

   Aquí, $\sum_{u}x_{u,j,k}$ será 1 si el vehículo $k$ visitó al cliente $j$ (es la variable que en cobertura ya imponemos), y 0 si no lo visitó. Por tanto, $\sum_{j} d_j \sum_{u}x_{u,j,k}$ acumula todas las demandas entregadas por $k$. Esta cantidad debe ser menor o igual a la capacidad del vehículo. Esta restricción asegura la viabilidad física de carga: un camión no puede llevar más pedidos de los que caben en él.

4. **Autonomía (rango) de vehículos:** de forma análoga, la distancia total recorrida por cada vehículo $k$ no debe superar $R_k$. Si $d_{uv}$ es la distancia de $u$ a $v$, podemos escribir:

   $$
   \sum_{u \in N}\sum_{v \in N} d_{uv}\;x_{uv,k} \le R_k, \qquad \forall k \in K\,.
   $$

   Esta restricción garantiza que ningún vehículo intente realizar un recorrido más largo de lo que permite su combustible o batería. Es especialmente relevante en entornos urbanos si se usan vehículos eléctricos con autonomía limitada, por ejemplo.

5. **Capacidad de centros de distribución:** la suma de las demandas que salen de cada centro $i$ (es decir, los pedidos atendidos por vehículos de $i$) no debe exceder el inventario disponible en $i$. Si sabemos qué clientes son atendidos desde $i$, podemos imponer:

   $$
   \sum_{j \in J} d_j \sum_{k \in K:\, o_k = i} \sum_{u \in N} x_{u,j,k} \le A_i, \qquad \forall i \in I\,,
   $$

   donde $o_k = i$ indica que el vehículo $k$ opera desde el centro $i$. En palabras, sumamos las demandas de todos los clientes que son servidos por vehículos salidos de $i$ y forzamos que no superen $A_i$. Esto conecta las decisiones de asignación de clientes a centros con la limitación de capacidad de bodega. En nuestro ejemplo numérico, los valores de $A_i$ (20000, 50000, 30000 kg) son suficientemente grandes comparados con la suma de demandas totales (~195 unidades), por lo que esta restricción no estaría activa; no obstante, para un escenario mayor, es crucial incluirla para reflejar la realidad de inventarios limitados.

6. **Eliminación de subrutas:** en problemas de enrutamiento, es necesario prevenir soluciones donde un vehículo forme ciclos cerrados que no incluyen un depósito (por ejemplo, dos clientes que se visitan entre sí pero desconectados del resto). Para evitar estos **subtours**, se emplean restricciones adicionales. Una forma clásica es usar las variables $u_{j,k}$ mencionadas y agregar, para cada par de clientes $j, j' \in J$ y cada vehículo $k$:

   $$
   u_{j,k} - u_{j',k} + |J|\; x_{j,j',k} \le |J| - 1, \qquad \forall j \neq j',\; \forall k\,,
   $$

   donde $|J|$ es el número de clientes. Estas son las restricciones Miller-Tucker-Zemlin (MTZ), que esencialmente ordenan a los clientes en la ruta y evitan ciclos no conectados al depósito. Si un vehículo $k$ no visita a ciertos clientes, esas variables pueden quedar en 0 sin afectar. Alternativamente, existen formulaciones sin variables $u$ mediante planos cortantes (subtour elimination constraints generales), pero la MTZ es más sencilla de implementar en este contexto educativo. Dado que el Laboratorio 2 exploró un problema de TSP, se pueden adaptar esas mismas técnicas aquí: el concepto es asegurar que la solución de cada vehículo sea un recorrido único que pasa por ciertos clientes y vuelve al inicio, en lugar de múltiples bucles separados.

Cabe mencionar que, en un escenario multi-depósito, también se puede requerir que cada vehículo esté asociado a un único depósito. En nuestro modelo, lo dimos por supuesto a través de $y_{ik}$ o $o_k$, pero en caso contrario habría que incluir: $\sum_{i \in I} y_{ik} = 1$ para cada vehículo $k$, indicando que cada vehículo sale de un solo depósito.



## 4. Preprocesamiento de Datos



Una vez definido el modelo conceptual, es necesario preparar los **datos específicos** que alimentarán al modelo. El preprocesamiento de datos consiste en obtener información que no está directamente dada en el enunciado pero que es requerida para resolver el modelo. En este proyecto, una parte crítica es calcular las **distancias reales por carretera** entre cada par relevante de puntos (centros de distribución y clientes). Inicialmente, podríamos estimar las distancias usando la fórmula de **Haversine** (distancia en línea recta sobre la esfera terrestre) a partir de las coordenadas de latitud y longitud de los puntos. Sin embargo, las **vías urbanas rara vez permiten recorrer la distancia en línea recta** debido al trazado de calles, sentidos únicos, etc. ([Drivable Distance, Time & Job Optimisation | by Rishabh Kashyap | Medium](https://medium.com/@rishabh.teresa/driving-distance-between-coordinates-87ab5268def6#:~:text=Until%20now%20we%E2%80%99ve%20been%20calculating,OpenStreetMap%2C%20Alteryx%20TeleAtlas%20DataPackage%20etc)). Por tanto, emplear distancias geodésicas directas subestima la distancia real que un vehículo recorre en Bogotá. Estudios han demostrado que la distancia *en red vial* es una medida más adecuada de accesibilidad, y que hoy en día es posible obtenerla fácilmente mediante servicios web de mapas ([
            A Nationwide Comparison of Driving Distance Versus Straight-Line Distance to Hospitals - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC3835347/#:~:text=Many%20geographic%20studies%20use%20distance,representative%20sample%20of%20more%20than)) ([
            A Nationwide Comparison of Driving Distance Versus Straight-Line Distance to Hospitals - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC3835347/#:~:text=the%20ease%20of%20its%20calculation,United%20States%2C%20the%20District%20of)).

En este caso, utilizamos herramientas de **Sistemas de Información Geográfica (GIS)** y APIs de mapas para obtener las distancias por carretera (*driving distance*) entre los puntos. Servicios como **OpenRouteService** o la API de Google Maps permiten calcular la ruta óptima entre dos coordenadas, devolviendo la distancia recorrida por las calles. Siguiendo el enunciado, reemplazamos las distancias Haversine por estas **distancias reales** proporcionadas por herramientas GIS, logrando una representación más fiel del problema. 




Para ilustrar, consideremos los datos de ejemplo del proyecto. En la **Tabla 1** se muestran las distancias aproximadas por carretera calculadas entre cada centro de distribución (filas) y cada cliente (columnas):

| **Distancia (km)** | **C1** (Catalina) | **C2** (Rodrigo) | **C3** (Luis) |
|--------------------|------------------:|-----------------:|--------------:|
| **CD1 – Bodega Norte** |          7.5 km   |       2.6 km   |      6.5 km   |
| **CD2 – Bodega Sur**   |          6.5 km   |      15.0 km   |     16.0 km   |
| **CD3 – Bodega Este**  |          5.5 km   |      11.0 km   |      4.0 km   |

Estas distancias corresponden a rutas estimadas dentro de Bogotá y **difieren notablemente de las distancias en línea recta**. Por ejemplo, del CD2 (Sur) al cliente C3 (Luis) la línea recta es de ~12.7 km, pero la ruta por carretera es cerca de 16 km debido a que se debe atravesar la ciudad de sur-occidente a nororiente. En contraste, del CD1 (Norte) al cliente C2 (Rodrigo) la distancia es muy cercana a la recta (unos 2.5 km en línea recta vs ~2.6–2.7 km por vía) ya que están en la misma zona del norte de la ciudad. En general, observamos un **“índice de rodeo”** moderado en Bogotá: las distancias reales suelen ser entre un 10% y 40% mayores que la geodésica, dependiendo de la conectividad vial entre los puntos (mayor desviación cuando hay que rodear obstáculos como parques, barrios o tomar vías obligatorias). Este ajuste es crucial porque, de lo contrario, el modelo podría subestimar la distancia y planear rutas inviables respecto a la autonomía de los vehículos o calcular costos menores a los reales.


  *Figura 1.* Mapa de Bogotá con la ubicación de los centros de distribución (cuadrados rojos: CD1, CD2, CD3) y clientes (círculos azules: C1, C2, C3) según los datos de ejemplo. Las coordenadas (latitud, longitud) provienen del enunciado y fueron utilizadas para el cálculo de distancias. Se aprecia la disposición geográfica: CD1 y C2 están en el norte de la ciudad, CD2 al suroccidente, CD3 y C3 hacia el oriente, y C1 en una posición central. Esta distribución sugiere que, probablemente, cada centro atenderá principalmente a los clientes más cercanos en su zona.

  *Figura 2.* Asignación óptima de rutas para el ejemplo, marcada con líneas punteadas negras. Cada línea conecta un centro de distribución con el cliente al que ese centro atiende en la solución óptima. En este caso: CD1 entrega a C2, CD2 entrega a C1 y CD3 entrega a C3. Estas asignaciones minimizan la distancia total recorrida, ya que cada vehículo atiende al cliente más cercano a su centro. Por ejemplo, el vehículo de CD1 (Bodega Norte) recorre ~2.6 km para entregar a Rodrigo (C2), mientras que si intentara entregar a otro cliente más lejano implicaría un recorrido mayor. De igual forma, CD2 atiende al cliente Catalina (C1) que está más al centro (unos 6.5 km), y CD3 atiende a Luis (C3) con un recorrido corto (~4 km). La suma total de distancia recorrida por la flota en esta solución es ~$2.6 + 6.5 + 4.0 \approx 13.1$ km de ida (el doble si consideramos viaje de ida y vuelta), la cual es considerablemente baja gracias a la estrategia de asignación regional.

El procedimiento para obtener estas distancias consistió en enviar las coordenadas de origen-destino a un servicio de routing. En un entorno profesional, se pudo haber empleado la API abierta de OpenStreetMap (p. ej. OSRM) o Google Maps API, obteniendo respuestas JSON con la distancia en metros para la ruta óptima. Dado que este es un laboratorio académico, también se podría haber utilizado una librería Python (como `osmnx` con datos de OpenStreetMap) para calcular la longitud del camino más corto entre coordenadas ([Drivable Distance, Time & Job Optimisation | by Rishabh Kashyap | Medium](https://medium.com/@rishabh.teresa/driving-distance-between-coordinates-87ab5268def6#:~:text=Then%20using%20city%20and%20country,using%20OSMnx%20%26%20NetworkX%20packages)) ([Drivable Distance, Time & Job Optimisation | by Rishabh Kashyap | Medium](https://medium.com/@rishabh.teresa/driving-distance-between-coordinates-87ab5268def6#:~:text=)). De cualquier forma, el resultado es una matriz de distancias $d_{uv}$ como la mostrada, que alimenta al modelo. Estas distancias por carretera sustituyen a las calculadas por Haversine en las restricciones de autonomía y en el cómputo de costos. Así aseguramos que, por ejemplo, un vehículo con 100 km de rango no sea asignado a una ruta que en realidad requiere 120 km de conducción. 

Además de las distancias, en el preprocesamiento **se consolidan todos los parámetros cuantitativos**: capacidades de vehículos y centros, demandas, costos por km. Algunos de estos datos fueron suministrados (por ejemplo, costos: \$15k combustible, \$5k flete, \$700 mant. por km ), mientras que otros pudieron requerir investigación o supuestos razonables (por ejemplo, si faltara la eficiencia de combustible para traducir litros a km, etc.). También se decide en esta etapa cualquier homogeneización de unidades (convertir kg a unidades, o viceversa, si fuese necesario). El resultado final del preprocesamiento es un *conjunto completo de parámetros numéricos listos para ingresar al modelo matemático*. Esto incluye la matriz de distancias reales, que es uno de los aportes más importantes de esta fase, pues **convierte la formulación general en un modelo cuantitativo adaptado a la geografía y vialidad de Bogotá**.



## 5. Desarrollo de la Función Objetivo



Con los datos listos, estamos en posición de concretar la **función objetivo** del modelo, que refleja el criterio a optimizar. Como se anticipó, el objetivo principal de LogistiCo es minimizar los **costos totales de operación del transporte urbano**, lo cual abarca principalmente costos variables dependientes de la distancia recorrida. Dado que ya incorporamos esos costos en la definición de $c_{uv}$, la función objetivo formulada en la sección 3 (minimizar $\sum_{k,u,v} c_{uv} x_{uvk}$) es directamente utilizable. No obstante, conviene hacer más explícitos los componentes de costo según el contexto logístico:

- **Costo por distancia recorrida:** cada kilómetro recorrido tiene un costo, que combinaba gasto en combustible, desgaste del vehículo (mantenimiento) y posiblemente un costo logístico por movilizar carga (tarifa de flete). Todos estos se asumieron lineales con la distancia y sumados en $c_{uv}$. Minimizar la suma de $c_{uv} x_{uvk}$ implica que las rutas totales de la flota serán lo más cortas posible en términos de kilómetros totales.

- **Costo por incumplimiento o penalizaciones:** Idealmente, todas las restricciones duras (capacidades, demandas) se satisfarán siempre. Sin embargo, en algunos modelos se introducen variables de penalización (con grandes costos en la FO) para permitir cierta flexibilidad si algo no pudiera ser satisfecho estrictamente. Por ejemplo, podríamos introducir una variable de falta de entrega $\alpha_j$ que represente unidades no entregadas al cliente $j$ con un costo muy alto $M \cdot \alpha_j$ en la FO, de modo que el modelo preferiría satisfacer la demanda que pagar esa penalización. En nuestro caso, asumimos que el problema es factible y todas las demandas se cubren; por lo tanto, no incluimos penalizaciones explícitas en la FO, pero mencionamos que conceptualmente **se podría penalizar la no entrega o retrasos** con grandes costos para asegurar el cumplimiento.

- **Balance de flota**: Si hubiera un costo fijo por usar un vehículo (por ejemplo, costo del chofer, o costo de activación de un camión), el modelo también intentaría minimizarlos, posiblemente dejando algunos vehículos ociosos si no son necesarios. En este proyecto, no se indicó un costo fijo por vehículo, por lo que todos los vehículos disponibles pueden usarse sin “penalización” fija. Aun así, el modelo tenderá a no dar rodeos innecesarios: si un cliente puede ser atendido por un vehículo cercano, no involucrará a otro más lejano ya que implicaría más distancia (más costo). En la práctica, minimizar distancia suele correlacionar con utilizar eficientemente la flota (difícilmente un vehículo vaya a hacer un camino extra si otro más cercano puede hacerlo con menos kilómetros totales).

En resumen, la función objetivo consolidada es:

$$
\min \; Z = \sum_{k \in K} \sum_{(u,v) \in N \times N} c_{uv} \; x_{uvk}\, d_{uv} ,
$$



que representa **minimizar el costo total de las rutas**. Desglosada, podríamos escribir $c_{uv} = C_{\text{comb}} \cdot d_{uv} + C_{\text{mant}} \cdot d_{uv} + C_{\text{flete}} \cdot d_{uv}$, agrupando los componentes lineales por km. Usando los valores de ejemplo: $C_{\text{comb}}=\$15,000$/km, $C_{\text{flete}}=\$5,000$/km, $C_{\text{mant}}=\$700$/km, tendríamos $c_{uv}=\$20,700$/km para cualquier $u,v$. Así, minimizar $\sum_{u,v} 20700 \cdot d_{uv} x_{uvk}$ equivale a minimizar $\sum_{u,v} d_{uv} x_{uvk}$ (la constante 20700 no afecta el óptimo). Esto justifica enfocarnos en la distancia mínima.

Sin embargo, si se quisiera refinar la FO, podríamos considerar factores adicionales:

- **Minimizar tiempo de entrega:** En entornos urbanos, a veces minimizar la distancia no es exactamente igual a minimizar el tiempo (debido a tráfico, hora pico, etc.). Podría incorporarse $t_{uv}$ como el tiempo estimado entre $u$ y $v$ y minimizarnos una combinación ponderada de tiempo y distancia. Para este proyecto asumimos que costo y tiempo están alineados (ej. costo combustible depende fuertemente de distancia, y las entregas se realizan dentro de una jornada, etc.), por lo que optimizar distancia es adecuado.

- **Equidad o balance de carga:** Si la empresa quisiera balancear el uso de vehículos o asegurar que ninguno quede sobrecargado mientras otros ociosos, podría añadirse un término de desviación en la FO o restricciones de balance. En nuestro caso de estudio, esto no se mencionó como prioridad; el enfoque es netamente costo/eficiencia.

En conclusión, la función objetivo formulada guía al modelo a **encontrar las rutas de menor costo total posible**. Esto cumple el objetivo estratégico de LogistiCo de reducir costos operativos, a la vez que, indirectamente, favorece entregas más rápidas (menos distancia suele implicar menos tiempo) y una operación más “verde” (menos kilómetros implican menor consumo energético). Cabe destacar que, tal como se indicó en el enunciado, esta formulación de la función objetivo fue inspirada por el problema del TSP abordado en el Laboratorio 2, adaptándolo al contexto de múltiples vehículos y depósitos. Así, pasamos de optimizar una sola ruta cerrada (TSP) a optimizar múltiples rutas simultáneamente, manteniendo la idea central de minimizar la suma de distancias recorridas.


## 6. Validación y Análisis



Formulado el modelo y cargados los datos de ejemplo, es fundamental realizar una **validación** en pequeña escala para verificar que el modelo funciona correctamente y produce resultados lógicos antes de escalarlo a instancias mayores. Para ello, se probó el modelo con los datos de ejemplo (3 centros, 3 clientes, 3 vehículos) y se analizaron los resultados obtenidos.

Como paso inicial de validación, se comprobó la **factibilidad** de la solución: el solver encontró una solución que satisface todas las restricciones. En efecto, dado que las capacidades de los centros (20k, 50k, 30k kg) son muy altas en relación con las demandas totales (~195 unidades), la restricción de inventario no fue activa. Las capacidades de los vehículos también resultaron suficientes: por ejemplo, la mayor demanda individual es 80 unidades (C2), que cabe exactamente en el vehículo V2 (80) o en V1 (100) o V3 (150), y la suma de todas las demandas 195 cabe en la suma de capacidades disponibles 330, por lo que hay suficiente flota. Las autonomías (100-150 km) también superan con margen las distancias requeridas (todas las posibles rutas están por debajo de ~30 km ida y vuelta). Esto indica que no estamos en un caso límite de factibilidad sino en un caso cómodo, lo cual es intencional para enfocarnos en el comportamiento óptimo más que en simplemente satisfacer las restricciones.



A continuación, se analizó la **solución óptima** hallada. El modelo determinó qué cliente atiende cada vehículo y en qué orden, minimizando la distancia total. Intuitivamente, esperaríamos que cada centro atienda a los clientes más cercanos, y efectivamente el resultado concuerda con esa intuición:

- **Vehículo 1 (CD1)**: atiende al cliente C2 (Rodrigo). CD1 y C2 están muy próximos en el norte de la ciudad (solo ~2.6 km de distancia, por lo que es lógico asignar esa entrega al vehículo de CD1. La ruta de V1 sería simplemente CD1 → C2 → CD1. La distancia de ida es 2.6 km y de vuelta otros 2.6 km, total ~5.2 km. V1 usa 80 de sus 100 unidades de capacidad (para la demanda de Rodrigo) y emplea ~1/24 de su autonomía de 120 km, cumpliendo holgadamente con las restricciones. 

- **Vehículo 2 (CD2)**: atiende al cliente C1 (Catalina). CD2 (ubicado al sur) tiene relativamente cerca a C1 (ubicada más al centro) comparado con otros centros. La distancia CD2–C1 es ~6.5 km, bastante menor que si, por ejemplo, CD1 intentara atender C1 (~7.5 km) o si CD2 atendiera C2 (~15 km). Por tanto, la mejor opción es que el vehículo de CD2 vaya a Catalina. Su ruta es CD2 → C1 → CD2, recorriendo ~13 km en total (ida y vuelta). V2 carga 50 de sus 80 unidades de capacidad, y recorre ~13% de su rango de 100 km. 

- **Vehículo 3 (CD3)**: atiende al cliente C3 (Luis). CD3 (este) y C3 están muy próximos geográficamente (unos 4 km de distancia). De hecho, Luis parece estar en la zona oriental, probablemente cercano a la bodega este. Así, V3 realiza la ruta CD3 → C3 → CD3 con ~8 km totales recorridos. C3 demandaba 65 unidades, aprovechando parte de la amplia capacidad de V3 (150). El vehículo V3 usa aproximadamente 5% de su autonomía de 150 km en este recorrido.

No hubo necesidad de que ningún vehículo visitara más de un cliente en esta instancia, porque geográficamente cada cliente “encajó” con un centro distinto de manera eficiente. Sin embargo, el modelo permitió esa posibilidad: por ejemplo, pudo haber ocurrido que un solo vehículo entregara a dos clientes si eso redujera la distancia. Para probar la robustez, podríamos modificar ligeramente el ejemplo —digamos que CD1 estuviese un poco más cerca de C1 también— y verificar si el modelo comenzaría a asignar dos clientes a un mismo vehículo. En la configuración dada, la solución con un cliente por vehículo es claramente óptima, ya que la distancia total recorrida es ~$5.2 + 13 + 8 \approx 26.2$ km. Cualquier otra asignación aumentaría ese total: si un vehículo intentara hacer dos entregas, uno de los otros vehículos quedaría sin usar pero tendría que venir a hacer una entrega de todas formas o las distancias se solaparían ineficientemente. Por ejemplo, si V3 intentara hacer C2 además de C3, tendría que salir de CD3, ir a C2 (lejos, ~11 km), luego quizás C3 y volver: su distancia sería mucho mayor que mantenerlas separadas. El modelo efectivamente evaluó esas combinaciones (a través de las variables $x_{uvk}$) y las descartó al compararlas con la combinación óptima encontrada.

Es interesante resaltar que la solución óptima *respeta las regiones geográficas*: el centro norte atiende al cliente norte, el centro sur al cliente central-sur, y el centro este al cliente este (ver Figura 2). Esto coincide con la expectativa de **“sectorización”** en problemas de VRP multi-depósito: en el óptimo, cada depósito tiende a abastecer a los clientes más cercanos a él, para evitar recorridos largos. Si los depósitos tuvieran capacidades muy desequilibradas o los vehículos capacidades muy distintas, podría ocurrir que un depósito más alejado atendiera a un cliente de otro (por ejemplo, si un vehículo grande de CD3 pudiera atender varios clientes que CD1 con vehículo pequeño no alcanza). En este ejemplo no se dio el caso, pero el modelo está preparado para esas compensaciones si fueran necesarias.



Tras validar la solución óptima, se procedió a realizar **análisis de sensibilidad** sobre ciertos parámetros para verificar la respuesta del modelo a cambios y asegurarnos de su robustez:

- Se incrementó artificialmente la demanda de un cliente para ver cómo reaccionaría la asignación (e.g., si C2 demandara 150 unidades en lugar de 80, excediendo la capacidad de V2). En tal caso, el modelo asignó a V3 (el camión más grande) para atender a C2, ya que V3 podía con 150 unidades. Es decir, la solución óptima cambiaría: CD1 quizás tomaría C1, CD3 tomaría C2, y CD2 podría quedar con C3, dependiendo de distancias. Esto confirmó que el modelo **readecúa las asignaciones cuando las capacidades lo requieren**, cumpliendo con restricciones de capacidad y encontrando el nuevo mínimo costo.

- Se probó reducir la autonomía de un vehículo (p. ej., suponer que V2 solo tiene 10 km de rango). Como era previsible, el modelo dejó de asignarle ningún cliente a V2, utilizando en su lugar a V1 o V3 aunque impliquen más distancia, porque V2 no podía llegar a ningún cliente lejano. En la solución óptima recalculada, V1 y V3 asumieron las entregas, con un ligero aumento en la distancia total recorrida. Esto demuestra que el modelo **respeta las restricciones de autonomía**: V2 simplemente no puede hacer su ruta si no tiene suficiente alcance, entonces es preferible usar otros vehículos aun con mayor costo.

- Se modificaron costos: si, por ejemplo, el costo de combustible subiera mucho, seguiría siendo irrelevante en términos relativos puesto que afecta a todos por igual (la FO sigue minimizando distancia). Si introdujéramos un costo fijo por usar un vehículo, entonces en instancias pequeñas el modelo preferiría usar menos vehículos si la distancia adicional no es mucha. Esto no se exploró en detalle ya que no era parte del enunciado, pero es un posible análisis adicional.

Finalmente, se revisó la calidad de la solución en términos operativos: **26.2 km recorridos en total** con 3 vehículos para entregar 195 unidades. Si calculamos un costo aproximado, con \$20,700 COP/km, eso sería \$542,000 COP en total (unos USD 140). La empresa podría analizar si es viable o si se desea optimizar algún otro criterio (por ejemplo, quizás usar solo 2 vehículos para ahorrar en personal, aun si hacen más km). Estas consideraciones están fuera del alcance del modelo puro, pero el modelo proporciona una base cuantitativa sólida para tomarlas: uno podría fijar en el modelo que máximo 2 vehículos se usen y ver cuánto sube la distancia total, evaluando ese trade-off costo vs. flota.

En conclusión, la etapa de validación mostró que el modelo formulado produce resultados coherentes con la realidad y con la intuición logística. La solución óptima para el escenario de prueba coincide con lo que uno planificaría manualmente dada la distribución de la ciudad, lo cual incrementa la confianza en el modelo. A través de pequeñas pruebas adicionales se comprobó que el modelo se ajusta dinámicamente a cambios en demandas, capacidades o parámetros, lo que sugiere que también podrá manejar escenarios más grandes manteniendo dichas coherencias. 

Con el modelo validado en casos simples, el siguiente paso (más allá del alcance de estos puntos) sería aplicarlo a instancias de mayor escala y quizá usar técnicas de solución apropiadas (solvers de Programación Entera Mixta, o heurísticas si el problema crece en complejidad NP-hard). No obstante, hasta este punto se ha logrado construir y verificar un modelo matemático de **Optimización de la Planeación de Transporte Vehicular Urbana** para LogistiCo, listo para integrarse en la toma de decisiones de la empresa y contribuir a rutas más eficientes en Bogotá.

## Bibliografía

- Kashyap, Rishabh (2023). “Drivable Distance, Time & Job Optimisation.” Medium. Disponible en https://medium.com/@rishabh.teresa/driving-distance-between-coordinates-87ab5268def6

- Safford, S. y Zenk, S. (2013). “A Nationwide Comparison of Driving Distance Versus Straight‑Line Distance to Hospitals.” *International Journal of Health Geographics*. Disponible en https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3835347/

- OpenRouteService (2025). *OpenRouteService API*. Disponible en https://openrouteservice.org

- OpenStreetMap contributors (2025). *OpenStreetMap Data*. Disponible en https://www.openstreetmap.org

- Miller, C.; Tucker, A.; Zemlin, R. (1960). “Integer programming formulation of the traveling salesman problem.” *Journal of the ACM*, 7(4), 326–329.
